### 4.2 模型参数的访问、初始化和共享

In [1]:
import torch
from torch import nn
from torch.nn import init

In [5]:
net = nn.Sequential(
    nn.Linear(4, 3), # pytorch默认初始化
    nn.ReLU(),
    nn.Linear(3, 1)
)

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()
print(Y.item())

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)
-0.5975306034088135


### 4.2.1 访问模型参数

> 1. **对于`Sequential`实例中含模型参数的层,可通过`Module`类的`parameters()`或者 `named_parameters()` 方法来访问所有参数(以迭代器形式)**
> 2. **`named_parameters()` 返回的名字自动加上了索引作为前缀**
> 3. **通过`net[index]`来访问网络任一层，此时返回的name没有索引前缀**
> 4. **param的类型`torch.nn.parameter.Parameter`,是`Tensor`的子类,如果一个`Tensor`是`Parameter`,那么会自动被添加到模型参数列表**
> 5. **`Parameter`是`Tensor`.可根据`data`访问参数值,`grad`访问梯度**

In [8]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [10]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [11]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        # weight1 是Parameter类型，会添加到网络参数中
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        # 而weight2却不会
        self.weight2 = torch.rand(20, 20)
    
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [14]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[ 0.3011,  0.4198,  0.3230,  0.2946],
        [ 0.2989,  0.3115, -0.3921,  0.2833],
        [ 0.2363,  0.0392,  0.2317, -0.2515]])
None
tensor([[ 0.6614,  0.4827,  0.4717,  0.2036],
        [-0.7584, -0.5535, -0.5410, -0.2335],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])
